In [1]:
import random
import imghdr
import shutil
import os
import tensorflow as tf
import pandas as pd

## Data preprocessing

In [2]:
os.chdir('/home/dell/Documentos/Untitled Folder/')

# lê arquivos .txt gerados pelo COVID-Net
train_labels = pd.read_csv("train_split.txt", sep=" ",header=None)
test_labels = pd.read_csv("test_split.txt", sep=" ",header=None)

# renomeia colunas

train_labels.columns = ['id', 'image', 'label', 'database']
test_labels.columns = ['id', 'image', 'label', 'database']


In [3]:
# mantendo as colunas que importam

train_labels = train_labels[['image','label']]
test_labels = test_labels[['image','label']]

train_labels.shape

(17226, 2)

In [6]:
# cria dataset de validação
seed(47)

val_labels = train_labels.sample(frac=0.2)
val_labels.shape

(3445, 2)

In [7]:
# separa treino de validação

new_train_image = []
new_train_label = []

for i in range(train_labels.shape[0]):
    
    if train_labels['image'][i] not in val_labels['image'].tolist():
        new_train_image.append(train_labels['image'][i])
        new_train_label.append(train_labels['label'][i])
        

new_train_dataset = pd.DataFrame()
new_train_dataset['image'] = new_train_image
new_train_dataset['label'] = new_train_label

train_labels = new_train_dataset.copy()
train_labels.shape

(13781, 2)

### Distribuição das classes

In [8]:
train_labels['label'].value_counts()

negative    12067
positive     1714
Name: label, dtype: int64

In [9]:
test_labels['label'].value_counts()

positive    291
negative    100
Name: label, dtype: int64

In [10]:
val_labels['label'].value_counts()

negative    3001
positive     444
Name: label, dtype: int64

## Movendo os arquivos para os diretórios corretos

In [ ]:
os.chdir('/home/dell/Documentos/COVID-Net/data/train/') # Diretório do treino gerado pelo COVID-Net

dir_train = '/home/dell/Documentos/Untitled Folder/data/train/' # diretório final treino
dir_test = '/home/dell/Documentos/Untitled Folder/data/test/' # diretório final teste
dir_val = '/home/dell/Documentos/Untitled Folder/data/val/' # diretório final validação

bad_list = []

# Copia treino e validação
for f in os.listdir():
    if str(f) in train_labels['image'].tolist():
        
        index_train = train_labels[train_labels['image']==f].index.values
        label = train_labels['label'].loc[index_train].values[0]
        shutil.copy(f, os.path.join(dir_train, label))
    
    elif str(f) in val_labels['image'].tolist():
        
        index_val = val_labels[val_labels['image']==f].index.values
        label = val_labels['label'].loc[index_val].values[0]
        shutil.copy(f, os.path.join(dir_val, label))
        

In [ ]:
os.chdir('/home/dell/Documentos/COVID-Net/data/test/')


# Copia teste

for f in os.listdir():
    if str(f) in test_labels['image'].tolist():
        
        index_test = test_labels[test_labels['image']==f].index.values
        label = test_labels['label'].loc[index_test].values[0]
        shutil.copy(f, os.path.join(dir_test, label))
    else:
        print('?')
        print(f)
        


In [13]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
test_dir = "/home/dell/Documentos/Untitled Folder/data/test/"

test_dataset = image_dataset_from_directory(test_dir,
                                                  shuffle=True,
                                                  labels="inferred",
                                                  label_mode="int",
                                                  batch_size=32,
                                                  image_size=(150, 150))


Found 391 files belonging to 2 classes.
